<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>SQL Notebook</font></h1>



## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Install & Import Everything Needed

In [31]:
# ---------------------------------------------
# Install and import all required libraries
# ---------------------------------------------
# (ipython-sql is already installed in most Jupyter setups)

import pandas as pd
from sqlalchemy import create_engine

# Load SQL magic extension
%load_ext sql

# Disable PrettyTable formatting (fixes JupyterLab 4 KeyError)
%config SqlMagic.displaycon = False
%config SqlMagic.autopandas = False


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Create Database Connection

In [32]:
# ---------------------------------------------
# Create a SQLite database and connect to it
# ---------------------------------------------
engine = create_engine("sqlite:///my_spacex.db")

# Register this connection for %sql magic
%sql sqlite:///my_spacex.db


## Load CSV and Create SQL Table

In [33]:
# ---------------------------------------------
# Load SpaceX CSV into pandas and write to SQL
# ---------------------------------------------
df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/"
    "IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
)

# Write raw table into SQL
df.to_sql("SPACEXTBL", engine, if_exists="replace", index=False, method="multi")

# Create cleaned table (Date NOT NULL)
%sql CREATE TABLE IF NOT EXISTS SPACEXTABLE AS \
     SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;


Done.


[]

## Run SQL Queries (Safe Display Using pandas)

In [34]:
# ---------------------------------------------
# Query 1: Distinct Launch Sites
# ---------------------------------------------
query1 = "SELECT DISTINCT Launch_Site FROM SPACEXTABLE;"
launch_sites = pd.read_sql(query1, engine)
launch_sites


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


## Rank Landing Outcomes

In [35]:
# ---------------------------------------------
# Query 2: Rank landing outcomes between dates
# ---------------------------------------------
query2 = """
SELECT 
    Landing_Outcome,
    COUNT(*) AS Outcome_Count
FROM 
    SPACEXTABLE
WHERE 
    Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY 
    Landing_Outcome
ORDER BY 
    Outcome_Count DESC;
"""

landing_outcomes = pd.read_sql(query2, engine)
landing_outcomes


,Landing_Outcome,Outcome_Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


## Display 5 records where launch sites begin with 'CCA'

In [36]:
query = """
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;
"""
pd.read_sql(query, engine)


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Display total payload mass carried by boosters launched by NASA (CRS)

In [37]:
query = """
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload
FROM SPACEXTABLE
WHERE Customer = 'NASA (CRS)';
"""
pd.read_sql(query, engine)


,Total_Payload
0,45596


## Display average payload mass carried by booster version 'F9 v1.1'

In [38]:
query = """
SELECT AVG(PAYLOAD_MASS__KG_) AS Avg_Payload
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1';
"""
pd.read_sql(query, engine)


,Avg_Payload
0,2928.4


## Date of first successful landing on ground pad

In [39]:
query = """
SELECT Date
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (ground pad)'
ORDER BY Date ASC
LIMIT 1;
"""
pd.read_sql(query, engine)


,Date
0,2015-12-22


## Names of boosters with success on drone ship and payload 4000–6000 kg

In [40]:
query = """
SELECT Booster_Version
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;
"""
pd.read_sql(query, engine)


,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


## Total number of successful and failed mission outcomes

In [41]:
query = """
SELECT Mission_Outcome, COUNT(*) AS Count
FROM SPACEXTABLE
GROUP BY Mission_Outcome;
"""
pd.read_sql(query, engine)


,Mission_Outcome,Count
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


## Booster versions that carried the maximum payload mass (subquery)

In [42]:
query = """
SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE
);
"""
pd.read_sql(query, engine)


,Booster_Version,PAYLOAD_MASS__KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


## Records showing month, failure landing outcomes (drone ship), booster version, launch site for 2015

In [43]:
query = """
SELECT 
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE 'Failure (drone ship)%'
  AND substr(Date, 1, 4) = '2015';
"""
pd.read_sql(query, engine)


,Month,Landing_Outcome,Booster_Version,Launch_Site
0,01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


## Rank landing outcomes between 2010‑06‑04 and 2017‑03‑20

In [44]:
query = """
SELECT 
    Landing_Outcome,
    COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;
"""
pd.read_sql(query, engine)


,Landing_Outcome,Outcome_Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
